In [13]:
#European Option
import math
import numpy as np
import numpy.random as npr
from pylab import plt,mpl


In [14]:
def gen_sn(M,I,anti_paths=True,mo_match = True):
    '''Function togenerate random number for simulation.
    
    Parameters
    M: number of intervals for discretization
    I: int number of paths to be simulated
    anti-paths: boolean
        use of antitethic variates
    mo_math:boolean
        use of moment matching
        '''
    
    if anti_paths is True:
        sn = npr.standard_normal((M+1,int(I/2)))
        sn = np.concatenate((sn,-sn),axis=1)
    else:
        sn = npr.standard_normal(( M+1 , I))
    if mo_match is True:
        sn = (sn-sn.mean())/sn.std()
    return sn

In [15]:
#European Option
S0= 100
r = 0.05
sigma = 0.25
T =1.0
I = 50000

In [16]:
def gbm_mcs_stat(K):
    '''Valuation o european call option using BlackScholesMerton model by montecarlo simulaion
    k:Strike Pirceof the option
    C0: pRESENT VALUE OF THE OPTIOn'''
    sn = gen_sn(1,I)
    #simulate index level at maturity
    ST = S0 * np.exp((r - 0.5*sigma**2)*T+ sigma*math.sqrt(T)*sn[1])
    #calculate the pay off maturity 
    hT = np.maximum(ST-K,0)
    #calculate MCS estimator
    C0 = math.exp(-r*T)*np.mean(hT)
    return ST,C0,hT

In [17]:
gbm_mcs_stat(K=110)

(array([ 93.14965592, 114.13197254, 124.26735017, ...,  98.07277556,
        134.27963232, 100.77674562]),
 8.076587345217755,
 array([ 0.        ,  4.13197254, 14.26735017, ...,  0.        ,
        24.27963232,  0.        ]))

In [18]:
#A more dynamic solutions for european call and  put option
M = 50
def gbm_mcs_call_put(K,option='call'):
    dt=T/M
    S= np.zeros((M+1,I))
    S[0] = 50
    sn = gen_sn(M,I)
    for t in range(1,M+1):
        S[t] = S[t-1]*np.exp((r-0.5*sigma**2)*dt + sigma*math.sqrt(dt)*sn[t])
        if option =='call':
            hT = np.maximum(S[-1]-K,0)
        else:
            hT= np.maximum(K-S[-1],0)
            #discounting Fv rates
    CO = math.exp(-r*T)*np.mean(hT)
    return S[-1],CO

In [19]:
gbm_mcs_call_put(K=40,option='call')

(array([59.03103259, 69.6521777 , 51.45538834, ..., 89.9541829 ,
        35.35486479, 27.35497241]),
 12.700346749574912)

In [20]:
gbm_mcs_call_put(K=40,option='put')

(array([73.11611724, 61.54603828, 54.29833185, ..., 36.81879116,
        56.24837162, 49.04441774]),
 0.7573473256488356)

In [21]:
#American Option
def gbm_mcs_amer(k,option='call'):
    dt = T/M
    df = math.exp(-r*dt)
    S = np.zeros((M+1,I))
    S[0] = S0
    
    sn = gen_sn(M,I)
    for t in range(1,M+1):
        S[t] = S[t-1] * np.exp((r-0.5*sigma**2)*dt + sigma*math.sqrt(dt)*sn[t])
    if option =='call':
        h = np.maximum(S-k,0)
    else:
        h = np.maximum(K-S,0)
    V = np.copy(h)
    for t in range (M-1,0,-1):
        reg = np.polyfit(S[t],V[t+1]*df,7)
        C = np.polyval(reg,S[t])
        V[t]=np.where(C>h[t],V[t+1]*df,h[t])
    C0 = df*np.mean(V[1])
    return C0

In [22]:
gbm_mcs_amer(40,option='call')

62.00345545403828

In [ ]:
euro_res = []
amer_res = []

k_list = np.arange(80,120,5)

for K in k_list:
    euro_res.append(gbm_mcs_call_put(K,'call'))
    amer_res.append(gbm_mcs_amer(K,'call'))
    